This project is an introduction to artificial neural nets: fully-connected
neural nets, hidden layers, activation functions, back-propagation, dropout.

### Implementation Steps:
1. **Setup:** Create a Jupyter notebook environment and import necessary libraries.
2. **Data Exploration and Preprocessing:** Load the dataset, perform exploratory data analysis, handle missing values, and perform feature engineering.
3. **Model Development:** Implement various models as specified in the mandatory part.
4. **Evaluation:** Compare models based on accuracy and AUC, and create a summary table.
5. **Optimization:** Work on the bonus part to enhance model performance.
6. **Finalization:** Prepare the submission file and documentation for peer review.

Tabular modeling takes data in the form of a table (like a spreadsheet or CSV). The objective is to predict the value in one column based on the values in the other columns.

## 1. Setup

In [1]:
%%capture
# ! pip install -Uqq fastbook dtreeviz

import fastbook
fastbook.setup_book()

import numpy as np
import pandas as pd

import numpy as np
import pandas as pd

from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
import dtreeviz
import pickle

from fastbook import *
from pandas.api.types import is_string_dtype, is_numeric_dtype, is_categorical_dtype
from fastai.tabular.all import *
from dtreeviz.trees import *
from IPython.display import Image, display_svg, SVG
random_state = 123
import warnings
warnings.filterwarnings('ignore')

In [3]:
def update_summary_table(approach, lib, accuracy, auc):

    baseline_index = summary_table[summary_table['Approach'] == approach].index

    summary_table.loc[baseline_index, 'Library Used'] = lib
    summary_table.loc[baseline_index, 'Accuracy'] = accuracy
    summary_table.loc[baseline_index, 'AUC Score'] = auc
    return summary_table

summary_table = pd.DataFrame({
    "Approach": ["Baseline (Naive Classifier)", "Random Forest", "Scikit-learn (MLPClassifier)", 
                 "Keras (TensorFlow)", "TensorFlow", "NumPy"],
    "Library Used": ["None", "Scikit-learn", "Scikit-learn", "TensorFlow", "TensorFlow", "NumPy"],
    "Algorithm": ["Naive Classifier", "Random Forest", "MLPClassifier", "Keras Neural Network", 
                  "TensorFlow Neural Network", "Custom Neural Network"],
    "Hyperparameters": ["N/A", "Optimized via Grid Search", "Default / Custom Settings", 
                        "Custom Settings", "Custom Settings", "Custom Implementation"],
    "Accuracy": ["To be filled", "To be filled", "To be filled", "To be filled", "To be filled", "To be filled"],
    "AUC Score": ["To be filled", "To be filled", "To be filled", "To be filled", "To be filled", "To be filled"]
})



summary_table

,Approach,Library Used,Algorithm,Hyperparameters,Accuracy,AUC Score
0,Baseline (Naive Classifier),None,Naive Classifier,N/A,To be filled,To be filled
1,Random Forest,Scikit-learn,Random Forest,Optimized via Grid Search,To be filled,To be filled
2,Scikit-learn (MLPClassifier),Scikit-learn,MLPClassifier,Default / Custom Settings,To be filled,To be filled
3,Keras (TensorFlow),TensorFlow,Keras Neural Network,Custom Settings,To be filled,To be filled
4,TensorFlow,TensorFlow,TensorFlow Neural Network,Custom Settings,To be filled,To be filled
5,NumPy,NumPy,Custom Neural Network,Custom Implementation,To be filled,To be filled


## 2. Data Exploration and Preprocessing

- Download and prepare the dataset.
- Perform data preprocessing, such as handling missing values and feature selection.
- Split the dataset into training and testing sets with stratification.

In tabular data some columns may contain numerical data, like "age," while others contain string values, like "sex." The numerical data can be directly fed to the model (with some optional preprocessing), but the other columns need to be converted to numbers. Since the values in those correspond to different categories, we often call this type of variables categorical variables. The first type are called continuous variables.

In [4]:
# file_path =  '/kaggle/input/bank-data-train-csv'
# df = pd.read_csv(f'{file_path}/bank_data_train.csv', low_memory=False)
file_path = 'data/bank_data_train.csv'
df = pd.read_csv(file_path, low_memory=False)

df.head()

,ID,CR_PROD_CNT_IL,AMOUNT_RUB_CLO_PRC,PRC_ACCEPTS_A_EMAIL_LINK,APP_REGISTR_RGN_CODE,PRC_ACCEPTS_A_POS,PRC_ACCEPTS_A_TK,TURNOVER_DYNAMIC_IL_1M,CNT_TRAN_AUT_TENDENCY1M,SUM_TRAN_AUT_TENDENCY1M,AMOUNT_RUB_SUP_PRC,PRC_ACCEPTS_A_AMOBILE,SUM_TRAN_AUT_TENDENCY3M,CLNT_TRUST_RELATION,PRC_ACCEPTS_TK,PRC_ACCEPTS_A_MTP,REST_DYNAMIC_FDEP_1M,CNT_TRAN_AUT_TENDENCY3M,CNT_ACCEPTS_TK,APP_MARITAL_STATUS,REST_DYNAMIC_SAVE_3M,CR_PROD_CNT_VCU,REST_AVG_CUR,CNT_TRAN_MED_TENDENCY1M,APP_KIND_OF_PROP_HABITATION,CLNT_JOB_POSITION_TYPE,AMOUNT_RUB_NAS_PRC,CLNT_JOB_POSITION,APP_DRIVING_LICENSE,TRANS_COUNT_SUP_PRC,APP_EDUCATION,CNT_TRAN_CLO_TENDENCY1M,SUM_TRAN_MED_TENDENCY1M,PRC_ACCEPTS_A_ATM,PRC_ACCEPTS_MTP,TRANS_COUNT_NAS_PRC,APP_TRAVEL_PASS,CNT_ACCEPTS_MTP,CR_PROD_CNT_TOVR,APP_CAR,CR_PROD_CNT_PIL,SUM_TRAN_CLO_TENDENCY1M,APP_POSITION_TYPE,TURNOVER_CC,TRANS_COUNT_ATM_PRC,AMOUNT_RUB_ATM_PRC,TURNOVER_PAYM,AGE,CNT_TRAN_MED_TENDENCY3M,CR_PROD_CNT_CC,SUM_TRAN_MED_TENDENCY3M,REST_DYNAMIC_FDEP_3M,REST_DYNAMIC_IL_1M,APP_EMP_TYPE,SUM_TRAN_CLO_TENDENCY3M,LDEAL_TENOR_MAX,LDEAL_YQZ_CHRG,CR_PROD_CNT_CCFP,DEAL_YQZ_IR_MAX,LDEAL_YQZ_COM,DEAL_YQZ_IR_MIN,CNT_TRAN_CLO_TENDENCY3M,REST_DYNAMIC_CUR_1M,REST_AVG_PAYM,LDEAL_TENOR_MIN,LDEAL_AMT_MONTH,APP_COMP_TYPE,LDEAL_GRACE_DAYS_PCT_MED,REST_DYNAMIC_CUR_3M,CNT_TRAN_SUP_TENDENCY3M,TURNOVER_DYNAMIC_CUR_1M,REST_DYNAMIC_PAYM_3M,SUM_TRAN_SUP_TENDENCY3M,REST_DYNAMIC_IL_3M,CNT_TRAN_ATM_TENDENCY3M,CNT_TRAN_ATM_TENDENCY1M,TURNOVER_DYNAMIC_IL_3M,SUM_TRAN_ATM_TENDENCY3M,DEAL_GRACE_DAYS_ACC_S1X1,AVG_PCT_MONTH_TO_PCLOSE,DEAL_YWZ_IR_MIN,SUM_TRAN_SUP_TENDENCY1M,DEAL_YWZ_IR_MAX,SUM_TRAN_ATM_TENDENCY1M,REST_DYNAMIC_PAYM_1M,CNT_TRAN_SUP_TENDENCY1M,DEAL_GRACE_DAYS_ACC_AVG,TURNOVER_DYNAMIC_CUR_3M,PACK,MAX_PCLOSE_DATE,LDEAL_YQZ_PC,CLNT_SETUP_TENOR,DEAL_GRACE_DAYS_ACC_MAX,TURNOVER_DYNAMIC_PAYM_3M,LDEAL_DELINQ_PER_MAXYQZ,TURNOVER_DYNAMIC_PAYM_1M,CLNT_SALARY_VALUE,TRANS_AMOUNT_TENDENCY3M,MED_DEBT_PRC_YQZ,TRANS_CNT_TENDENCY3M,LDEAL_USED_AMT_AVG_YQZ,REST_DYNAMIC_CC_1M,LDEAL_USED_AMT_AVG_YWZ,TURNOVER_DYNAMIC_CC_1M,AVG_PCT_DEBT_TO_DEAL_AMT,LDEAL_ACT_DAYS_ACC_PCT_AVG,REST_DYNAMIC_CC_3M,MED_DEBT_PRC_YWZ,LDEAL_ACT_DAYS_PCT_TR3,LDEAL_ACT_DAYS_PCT_AAVG,LDEAL_DELINQ_PER_MAXYWZ,TURNOVER_DYNAMIC_CC_3M,LDEAL_ACT_DAYS_PCT_TR,LDEAL_ACT_DAYS_PCT_TR4,LDEAL_ACT_DAYS_PCT_CURR,TARGET
0,146841,0,0.000000,NaN,NaN,NaN,NaN,0.0,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.541683,0,156067.339767,NaN,NaN,NaN,0.0,начальник отдела,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,0,NaN,0,NaN,NaN,0.0,1.000000,1.000000,0.0,660,NaN,0,NaN,0.0,0.0,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,0.134651,0.0,NaN,NaN,NaN,0.0,0.474134,NaN,0.131910,0.0,NaN,0.0,0.406780,0.101695,0.0,0.483032,NaN,NaN,NaN,NaN,NaN,0.134634,0.0,NaN,NaN,0.442285,K01,NaN,NaN,1.593023,NaN,0.0,NaN,0.0,NaN,0.483032,NaN,0.406780,NaN,0.0,NaN,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0
1,146842,0,0.041033,NaN,NaN,NaN,NaN,0.0,0.166667,0.186107,0.244678,NaN,0.670968,NaN,NaN,NaN,0.0,0.666667,NaN,NaN,0.000000,0,4278.845817,NaN,NaN,NaN,0.0,NaN,NaN,0.454545,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,0,NaN,0,NaN,NaN,0.0,0.109091,0.410691,0.0,552,NaN,0,NaN,0.0,0.0,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,0.239365,0.0,NaN,NaN,NaN,0.0,0.384264,0.6,0.101934,0.0,0.510712,0.0,0.333333,0.166667,0.0,0.200000,NaN,NaN,NaN,0.309799,NaN,0.133333,0.0,0.24,NaN,0.515876,102,NaN,NaN,1.587647,NaN,0.0,NaN,0.0,NaN,0.394340,NaN,0.545455,NaN,0.0,NaN,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0
2,146843,0,0.006915,0.0,NaN,0.0,0.0,0.0,NaN,NaN,0.000000,0.0,NaN,NaN,0.0,0.0,0.0,NaN,0.0,NaN,0.000000,0,112837.062817,NaN,NaN,NaN,0.0,ГЕНЕРАЛЬНЫЙ ДИРЕКТОР,NaN,0.000000,NaN,NaN,NaN,0.0,0.0,0.0,NaN,0.0,0,NaN,0,NaN,NaN,0.0,0.810811,0.925140,0.0,420,NaN,0,NaN,0.0,0.0,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,0.084341,0.0,NaN,NaN,NaN,0.0,0.336136,NaN,0.121041,0.0,NaN,0.0,0.366667,0.133333,0.0,0.431656,NaN,NaN,NaN,NaN,NaN,0.063129,0.0,NaN,NaN,0.522833,102,NaN,NaN,1.587647,NaN,0.0,NaN,0.0,NaN,0.399342,NaN,0.297297,NaN,0.0,NaN,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0
3,146844,0,0.00

In [5]:
len(df.columns)

116

In [6]:
constant_or_nan_cols = [col for col in df.columns if df[col].nunique(dropna=True) <= 1]
df.drop(columns=constant_or_nan_cols, inplace=True)
constant_or_nan_cols

['PRC_ACCEPTS_A_EMAIL_LINK',
 'PRC_ACCEPTS_A_POS',
 'PRC_ACCEPTS_A_TK',
 'PRC_ACCEPTS_A_AMOBILE',
 'PRC_ACCEPTS_TK',
 'PRC_ACCEPTS_A_MTP',
 'CNT_ACCEPTS_TK',
 'PRC_ACCEPTS_A_ATM',
 'PRC_ACCEPTS_MTP',
 'CNT_ACCEPTS_MTP']

In [7]:
len(df.columns)

106

In [8]:
df[df.select_dtypes(include=["object"]).columns] = df.select_dtypes(include=["object"]).apply(lambda c: c.str.lower())

In [9]:
df

,ID,CR_PROD_CNT_IL,AMOUNT_RUB_CLO_PRC,APP_REGISTR_RGN_CODE,TURNOVER_DYNAMIC_IL_1M,CNT_TRAN_AUT_TENDENCY1M,SUM_TRAN_AUT_TENDENCY1M,AMOUNT_RUB_SUP_PRC,SUM_TRAN_AUT_TENDENCY3M,CLNT_TRUST_RELATION,REST_DYNAMIC_FDEP_1M,CNT_TRAN_AUT_TENDENCY3M,APP_MARITAL_STATUS,REST_DYNAMIC_SAVE_3M,CR_PROD_CNT_VCU,REST_AVG_CUR,CNT_TRAN_MED_TENDENCY1M,APP_KIND_OF_PROP_HABITATION,CLNT_JOB_POSITION_TYPE,AMOUNT_RUB_NAS_PRC,CLNT_JOB_POSITION,APP_DRIVING_LICENSE,TRANS_COUNT_SUP_PRC,APP_EDUCATION,CNT_TRAN_CLO_TENDENCY1M,SUM_TRAN_MED_TENDENCY1M,TRANS_COUNT_NAS_PRC,APP_TRAVEL_PASS,CR_PROD_CNT_TOVR,APP_CAR,CR_PROD_CNT_PIL,SUM_TRAN_CLO_TENDENCY1M,APP_POSITION_TYPE,TURNOVER_CC,TRANS_COUNT_ATM_PRC,AMOUNT_RUB_ATM_PRC,TURNOVER_PAYM,AGE,CNT_TRAN_MED_TENDENCY3M,CR_PROD_CNT_CC,SUM_TRAN_MED_TENDENCY3M,REST_DYNAMIC_FDEP_3M,REST_DYNAMIC_IL_1M,APP_EMP_TYPE,SUM_TRAN_CLO_TENDENCY3M,LDEAL_TENOR_MAX,LDEAL_YQZ_CHRG,CR_PROD_CNT_CCFP,DEAL_YQZ_IR_MAX,LDEAL_YQZ_COM,DEAL_YQZ_IR_MIN,CNT_TRAN_CLO_TENDENCY3M,REST_DYNAMIC_CUR_1M,REST_AVG_PAYM,LDEAL_TENOR_MIN,LDEAL_AMT_MONTH,APP_COMP_TYPE,LDEAL_GRACE_DAYS_PCT_MED,REST_DYNAMIC_CUR_3M,CNT_TRAN_SUP_TENDENCY3M,TURNOVER_DYNAMIC_CUR_1M,REST_DYNAMIC_PAYM_3M,SUM_TRAN_SUP_TENDENCY3M,REST_DYNAMIC_IL_3M,CNT_TRAN_ATM_TENDENCY3M,CNT_TRAN_ATM_TENDENCY1M,TURNOVER_DYNAMIC_IL_3M,SUM_TRAN_ATM_TENDENCY3M,DEAL_GRACE_DAYS_ACC_S1X1,AVG_PCT_MONTH_TO_PCLOSE,DEAL_YWZ_IR_MIN,SUM_TRAN_SUP_TENDENCY1M,DEAL_YWZ_IR_MAX,SUM_TRAN_ATM_TENDENCY1M,REST_DYNAMIC_PAYM_1M,CNT_TRAN_SUP_TENDENCY1M,DEAL_GRACE_DAYS_ACC_AVG,TURNOVER_DYNAMIC_CUR_3M,PACK,MAX_PCLOSE_DATE,LDEAL_YQZ_PC,CLNT_SETUP_TENOR,DEAL_GRACE_DAYS_ACC_MAX,TURNOVER_DYNAMIC_PAYM_3M,LDEAL_DELINQ_PER_MAXYQZ,TURNOVER_DYNAMIC_PAYM_1M,CLNT_SALARY_VALUE,TRANS_AMOUNT_TENDENCY3M,MED_DEBT_PRC_YQZ,TRANS_CNT_TENDENCY3M,LDEAL_USED_AMT_AVG_YQZ,REST_DYNAMIC_CC_1M,LDEAL_USED_AMT_AVG_YWZ,TURNOVER_DYNAMIC_CC_1M,AVG_PCT_DEBT_TO_DEAL_AMT,LDEAL_ACT_DAYS_ACC_PCT_AVG,REST_DYNAMIC_CC_3M,MED_DEBT_PRC_YWZ,LDEAL_ACT_DAYS_PCT_TR3,LDEAL_ACT_DAYS_PCT_AAVG,LDEAL_DELINQ_PER_MAXYWZ,TURNOVER_DYNAMIC_CC_3M,LDEAL_ACT_DAYS_PCT_TR,LDEAL_ACT_DAYS_PCT_TR4,LDEAL_ACT_DAYS_PCT_CURR,TARGET
0,146841,0,0.000000,NaN,0.0,NaN,NaN,0.000000,NaN,NaN,0.0,NaN,NaN,0.541683,0,156067.339767,NaN,NaN,NaN,0.000000,начальник отдела,NaN,0.000000,NaN,NaN,NaN,0.000000,NaN,0,NaN,0,NaN,NaN,0.0,1.000000,1.000000,0.0,660,NaN,0,NaN,0.0,0.0,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,0.134651,0.0,NaN,NaN,NaN,0.0,0.474134,NaN,0.131910,0.0,NaN,0.0,0.406780,0.101695,0.0,0.483032,NaN,NaN,NaN,NaN,NaN,0.134634,0.0,NaN,NaN,0.442285,k01,NaN,NaN,1.593023,NaN,0.0,NaN,0.0,NaN,0.483032,NaN,0.406780,NaN,0.0,NaN,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0
1,146842,0,0.041033,NaN,0.0,0.166667,0.186107,0.244678,0.670968,NaN,0.0,0.666667,NaN,0.000000,0,4278.845817,NaN,NaN,NaN,0.000000,NaN,NaN,0.454545,NaN,NaN,NaN,0.000000,NaN,0,NaN,0,NaN,NaN,0.0,0.109091,0.410691,0.0,552,NaN,0,NaN,0.0,0.0,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,0.239365,0.0,NaN,NaN,NaN,0.0,0.384264,0.600000,0.101934,0.0,0.510712,0.0,0.333333,0.166667,0.0,0.200000,NaN,NaN,NaN,0.309799,NaN,0.133333,0.0,0.240000,NaN,0.515876,102,NaN,NaN,1.587647,NaN,0.0,NaN,0.0,NaN,0.394340,NaN,0.545455,NaN,0.0,NaN,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0
2,146843,0,0.006915,NaN,0.0,NaN,NaN,0.000000,NaN,NaN,0.0,NaN,NaN,0.000000,0,112837.062817,NaN,NaN,NaN,0.000000,генеральный директор,NaN,0.000000,NaN,NaN,NaN,0.000000,NaN,0,NaN,0,NaN,NaN,0.0,0.810811,0.925140,0.0,420,NaN,0,NaN,0.0,0.0,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,0.084341,0.0,NaN,NaN,NaN,0.0,0.336136,NaN,0.121041,0.0,NaN,0.0,0.366667,0.133333,0.0,0.431656,NaN,NaN,NaN,NaN,NaN,0.063129,0.0,NaN,NaN,0.522833,102,NaN,NaN,1.587647,NaN,0.0,NaN,0.0,NaN,0.399342,NaN,0.297297,NaN,0.0,NaN,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0
3,146844,0,0.000000,NaN,0.0,NaN,NaN,0.000000,NaN,NaN,0.0,NaN,NaN,0.005874,0,42902.902883,NaN,NaN,NaN,0.000000,менеджер иао,NaN,0.000000,NaN,NaN,NaN,0.000000,NaN,0,NaN,0,NaN,NaN,0.0,1.000000,1.000000,0.0,372,NaN,0,NaN,0.0,0.0,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,0.005659,0.0,NaN,

In [10]:
translation_dict = {
    'близкий ро': 'close relative', 'друг': 'friend', 'отец': 'father',
    'сестра': 'sister', 'сын': 'son', 'мать': 'mother', 'муж': 'husband',
    'брат': 'brother', 'дальний ро': 'distant relative', 'дочь': 'daughter',
    'жена': 'wife', 'mother': 'mother', 'brother': 'brother', 'friend': 'friend',
    'sister': 'sister', 'other': 'other', 'relative': 'relative', 'daughter': 'daughter',
    'son': 'son', 'father': 'father'
}

df['CLNT_TRUST_RELATION'] = df['CLNT_TRUST_RELATION'].map(translation_dict)
df['CLNT_TRUST_RELATION'].unique()

array([nan, 'mother', 'brother', 'friend', 'sister', 'other', 'relative', 'daughter', 'son', 'father', 'close relative', 'husband', 'distant relative', 'wife'], dtype=object)

In [11]:
majority_class = df['TARGET'].value_counts().idxmax()
majority_class

0

In [12]:
# Filter the DataFrame to keep only rows with the minority class
# df_filtered = df[df['TARGET'] != majority_class]
df_filtered_0 = df[df['TARGET'] == majority_class]
df_filtered_0['TARGET'].value_counts()

TARGET
0    326265
Name: count, dtype: int64

In [13]:
df_filtered_0.shape

(326265, 106)

In [14]:
threshold = 52  # Adjust this threshold based on your requirements
df_filtered_0 = df_filtered_0.dropna(thresh=df_filtered_0.shape[1] - threshold + 1)
df_filtered_0

,ID,CR_PROD_CNT_IL,AMOUNT_RUB_CLO_PRC,APP_REGISTR_RGN_CODE,TURNOVER_DYNAMIC_IL_1M,CNT_TRAN_AUT_TENDENCY1M,SUM_TRAN_AUT_TENDENCY1M,AMOUNT_RUB_SUP_PRC,SUM_TRAN_AUT_TENDENCY3M,CLNT_TRUST_RELATION,REST_DYNAMIC_FDEP_1M,CNT_TRAN_AUT_TENDENCY3M,APP_MARITAL_STATUS,REST_DYNAMIC_SAVE_3M,CR_PROD_CNT_VCU,REST_AVG_CUR,CNT_TRAN_MED_TENDENCY1M,APP_KIND_OF_PROP_HABITATION,CLNT_JOB_POSITION_TYPE,AMOUNT_RUB_NAS_PRC,CLNT_JOB_POSITION,APP_DRIVING_LICENSE,TRANS_COUNT_SUP_PRC,APP_EDUCATION,CNT_TRAN_CLO_TENDENCY1M,SUM_TRAN_MED_TENDENCY1M,TRANS_COUNT_NAS_PRC,APP_TRAVEL_PASS,CR_PROD_CNT_TOVR,APP_CAR,CR_PROD_CNT_PIL,SUM_TRAN_CLO_TENDENCY1M,APP_POSITION_TYPE,TURNOVER_CC,TRANS_COUNT_ATM_PRC,AMOUNT_RUB_ATM_PRC,TURNOVER_PAYM,AGE,CNT_TRAN_MED_TENDENCY3M,CR_PROD_CNT_CC,SUM_TRAN_MED_TENDENCY3M,REST_DYNAMIC_FDEP_3M,REST_DYNAMIC_IL_1M,APP_EMP_TYPE,SUM_TRAN_CLO_TENDENCY3M,LDEAL_TENOR_MAX,LDEAL_YQZ_CHRG,CR_PROD_CNT_CCFP,DEAL_YQZ_IR_MAX,LDEAL_YQZ_COM,DEAL_YQZ_IR_MIN,CNT_TRAN_CLO_TENDENCY3M,REST_DYNAMIC_CUR_1M,REST_AVG_PAYM,LDEAL_TENOR_MIN,LDEAL_AMT_MONTH,APP_COMP_TYPE,LDEAL_GRACE_DAYS_PCT_MED,REST_DYNAMIC_CUR_3M,CNT_TRAN_SUP_TENDENCY3M,TURNOVER_DYNAMIC_CUR_1M,REST_DYNAMIC_PAYM_3M,SUM_TRAN_SUP_TENDENCY3M,REST_DYNAMIC_IL_3M,CNT_TRAN_ATM_TENDENCY3M,CNT_TRAN_ATM_TENDENCY1M,TURNOVER_DYNAMIC_IL_3M,SUM_TRAN_ATM_TENDENCY3M,DEAL_GRACE_DAYS_ACC_S1X1,AVG_PCT_MONTH_TO_PCLOSE,DEAL_YWZ_IR_MIN,SUM_TRAN_SUP_TENDENCY1M,DEAL_YWZ_IR_MAX,SUM_TRAN_ATM_TENDENCY1M,REST_DYNAMIC_PAYM_1M,CNT_TRAN_SUP_TENDENCY1M,DEAL_GRACE_DAYS_ACC_AVG,TURNOVER_DYNAMIC_CUR_3M,PACK,MAX_PCLOSE_DATE,LDEAL_YQZ_PC,CLNT_SETUP_TENOR,DEAL_GRACE_DAYS_ACC_MAX,TURNOVER_DYNAMIC_PAYM_3M,LDEAL_DELINQ_PER_MAXYQZ,TURNOVER_DYNAMIC_PAYM_1M,CLNT_SALARY_VALUE,TRANS_AMOUNT_TENDENCY3M,MED_DEBT_PRC_YQZ,TRANS_CNT_TENDENCY3M,LDEAL_USED_AMT_AVG_YQZ,REST_DYNAMIC_CC_1M,LDEAL_USED_AMT_AVG_YWZ,TURNOVER_DYNAMIC_CC_1M,AVG_PCT_DEBT_TO_DEAL_AMT,LDEAL_ACT_DAYS_ACC_PCT_AVG,REST_DYNAMIC_CC_3M,MED_DEBT_PRC_YWZ,LDEAL_ACT_DAYS_PCT_TR3,LDEAL_ACT_DAYS_PCT_AAVG,LDEAL_DELINQ_PER_MAXYWZ,TURNOVER_DYNAMIC_CC_3M,LDEAL_ACT_DAYS_PCT_TR,LDEAL_ACT_DAYS_PCT_TR4,LDEAL_ACT_DAYS_PCT_CURR,TARGET
1,146842,0,0.041033,NaN,0.0,0.166667,0.186107,0.244678,0.670968,NaN,0.0,0.666667,NaN,0.0,0,4278.845817,NaN,NaN,NaN,0.000000,NaN,NaN,0.454545,NaN,NaN,NaN,0.000000,NaN,0,NaN,0,NaN,NaN,0.0,0.109091,0.410691,0.000000,552,NaN,0,NaN,0.0,0.0,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,0.239365,0.000000,NaN,NaN,NaN,0.0,0.384264,0.600000,0.101934,0.000000,0.510712,0.0,0.333333,0.166667,0.0,0.200000,NaN,NaN,NaN,0.309799,NaN,0.133333,0.000000,0.240000,NaN,0.515876,102,NaN,NaN,1.587647,NaN,0.000000,NaN,0.000000,NaN,0.394340,NaN,0.545455,NaN,0.0,NaN,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0
5,146846,0,0.077010,NaN,0.0,0.500000,0.098136,0.050708,1.000000,NaN,0.0,1.000000,NaN,0.0,0,0.000000,NaN,NaN,NaN,0.008501,NaN,NaN,0.207547,NaN,0.250000,NaN,0.113208,NaN,0,NaN,0,0.345196,NaN,0.0,0.226415,0.767778,248361.558333,384,NaN,0,NaN,0.0,0.0,NaN,0.649373,NaN,NaN,0,NaN,NaN,NaN,0.500000,0.000000,105180.487383,NaN,NaN,NaN,0.0,0.000000,0.727273,0.000000,0.657494,0.627798,0.0,0.500000,0.166667,0.0,0.854130,NaN,NaN,NaN,0.110611,NaN,0.018453,0.023562,0.181818,NaN,0.000000,105,NaN,NaN,1.593023,NaN,0.700485,NaN,0.033022,NaN,0.780661,NaN,0.698113,NaN,0.0,NaN,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0
8,146850,0,0.017322,NaN,0.0,0.333333,0.357143,0.002935,0.357143,NaN,0.0,0.333333,NaN,0.0,0,116791.222983,NaN,NaN,NaN,0.000000,генеральный директор,NaN,0.020408,NaN,NaN,NaN,0.000000,NaN,2,NaN,0,NaN,NaN,0.0,0.448980,0.751444,0.000000,492,NaN,0,NaN,0.0,0.0,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,0.707973,0.000000,NaN,NaN,NaN,0.0,0.771877,NaN,0.911780,0.000000,NaN,0.0,0.272727,0.045455,0.0,0.353191,0.0,NaN,20.00,NaN,45.00,0.015648,0.000000,NaN,0.0,0.937303,o01,NaN,NaN,1.590335,0.0,0.000000,NaN,0.000000,NaN,0.297416,NaN,0.326531,NaN,0.0,1.0,0.0,NaN,0.001976,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
9,146851,0,0.067036,NaN,0.0,NaN,NaN,0.426053,NaN,NaN,0.0,NaN,NaN,0.0,0,51312.385183,0.300,NaN,NaN,0.000000,архитектор,Na

In [15]:
df_filtered_1 = df[df['TARGET'] != majority_class]
df_filtered_1

,ID,CR_PROD_CNT_IL,AMOUNT_RUB_CLO_PRC,APP_REGISTR_RGN_CODE,TURNOVER_DYNAMIC_IL_1M,CNT_TRAN_AUT_TENDENCY1M,SUM_TRAN_AUT_TENDENCY1M,AMOUNT_RUB_SUP_PRC,SUM_TRAN_AUT_TENDENCY3M,CLNT_TRUST_RELATION,REST_DYNAMIC_FDEP_1M,CNT_TRAN_AUT_TENDENCY3M,APP_MARITAL_STATUS,REST_DYNAMIC_SAVE_3M,CR_PROD_CNT_VCU,REST_AVG_CUR,CNT_TRAN_MED_TENDENCY1M,APP_KIND_OF_PROP_HABITATION,CLNT_JOB_POSITION_TYPE,AMOUNT_RUB_NAS_PRC,CLNT_JOB_POSITION,APP_DRIVING_LICENSE,TRANS_COUNT_SUP_PRC,APP_EDUCATION,CNT_TRAN_CLO_TENDENCY1M,SUM_TRAN_MED_TENDENCY1M,TRANS_COUNT_NAS_PRC,APP_TRAVEL_PASS,CR_PROD_CNT_TOVR,APP_CAR,CR_PROD_CNT_PIL,SUM_TRAN_CLO_TENDENCY1M,APP_POSITION_TYPE,TURNOVER_CC,TRANS_COUNT_ATM_PRC,AMOUNT_RUB_ATM_PRC,TURNOVER_PAYM,AGE,CNT_TRAN_MED_TENDENCY3M,CR_PROD_CNT_CC,SUM_TRAN_MED_TENDENCY3M,REST_DYNAMIC_FDEP_3M,REST_DYNAMIC_IL_1M,APP_EMP_TYPE,SUM_TRAN_CLO_TENDENCY3M,LDEAL_TENOR_MAX,LDEAL_YQZ_CHRG,CR_PROD_CNT_CCFP,DEAL_YQZ_IR_MAX,LDEAL_YQZ_COM,DEAL_YQZ_IR_MIN,CNT_TRAN_CLO_TENDENCY3M,REST_DYNAMIC_CUR_1M,REST_AVG_PAYM,LDEAL_TENOR_MIN,LDEAL_AMT_MONTH,APP_COMP_TYPE,LDEAL_GRACE_DAYS_PCT_MED,REST_DYNAMIC_CUR_3M,CNT_TRAN_SUP_TENDENCY3M,TURNOVER_DYNAMIC_CUR_1M,REST_DYNAMIC_PAYM_3M,SUM_TRAN_SUP_TENDENCY3M,REST_DYNAMIC_IL_3M,CNT_TRAN_ATM_TENDENCY3M,CNT_TRAN_ATM_TENDENCY1M,TURNOVER_DYNAMIC_IL_3M,SUM_TRAN_ATM_TENDENCY3M,DEAL_GRACE_DAYS_ACC_S1X1,AVG_PCT_MONTH_TO_PCLOSE,DEAL_YWZ_IR_MIN,SUM_TRAN_SUP_TENDENCY1M,DEAL_YWZ_IR_MAX,SUM_TRAN_ATM_TENDENCY1M,REST_DYNAMIC_PAYM_1M,CNT_TRAN_SUP_TENDENCY1M,DEAL_GRACE_DAYS_ACC_AVG,TURNOVER_DYNAMIC_CUR_3M,PACK,MAX_PCLOSE_DATE,LDEAL_YQZ_PC,CLNT_SETUP_TENOR,DEAL_GRACE_DAYS_ACC_MAX,TURNOVER_DYNAMIC_PAYM_3M,LDEAL_DELINQ_PER_MAXYQZ,TURNOVER_DYNAMIC_PAYM_1M,CLNT_SALARY_VALUE,TRANS_AMOUNT_TENDENCY3M,MED_DEBT_PRC_YQZ,TRANS_CNT_TENDENCY3M,LDEAL_USED_AMT_AVG_YQZ,REST_DYNAMIC_CC_1M,LDEAL_USED_AMT_AVG_YWZ,TURNOVER_DYNAMIC_CC_1M,AVG_PCT_DEBT_TO_DEAL_AMT,LDEAL_ACT_DAYS_ACC_PCT_AVG,REST_DYNAMIC_CC_3M,MED_DEBT_PRC_YWZ,LDEAL_ACT_DAYS_PCT_TR3,LDEAL_ACT_DAYS_PCT_AAVG,LDEAL_DELINQ_PER_MAXYWZ,TURNOVER_DYNAMIC_CC_3M,LDEAL_ACT_DAYS_PCT_TR,LDEAL_ACT_DAYS_PCT_TR4,LDEAL_ACT_DAYS_PCT_CURR,TARGET
26,146871,0,0.020267,NaN,0.0,1.0,1.000000,0.111424,1.000000,mother,0.0,1.000000,NaN,0.994683,0,1049.544317,NaN,NaN,NaN,0.000000,NaN,NaN,0.379310,NaN,NaN,NaN,0.000000,NaN,1,NaN,0,NaN,NaN,0.0,0.000000,0.000000,0.000000,624,NaN,0,NaN,0.0,0.0,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,0.548043,0.000000,NaN,NaN,NaN,0.0,0.597867,0.363636,0.375444,0.000000,0.218070,0.0,NaN,NaN,0.0,NaN,0.0,NaN,20.00,0.218070,20.0,NaN,0.000000,0.363636,0.0,0.787479,o01,NaN,NaN,2.101087,0.0,0.000000,NaN,0.000000,NaN,0.655993,NaN,0.551724,NaN,0.0,1.0,0.0,NaN,0.479087,0.0,1.0,0.615412,0.615412,0.5,0.0,0.233333,0.233333,0.233333,1
27,146872,0,0.000000,NaN,0.0,NaN,NaN,0.016017,NaN,friend,0.0,NaN,NaN,0.000000,0,372.021867,NaN,NaN,NaN,0.000000,водитель-экспедитор,NaN,0.333333,NaN,NaN,NaN,0.000000,NaN,2,NaN,0,NaN,NaN,0.0,0.666667,0.983983,0.000000,276,NaN,0,NaN,0.0,0.0,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,0.508451,0.000000,NaN,NaN,NaN,0.0,0.929992,1.000000,0.366414,0.000000,1.000000,0.0,0.900000,0.500000,0.0,0.975369,0.0,NaN,45.00,1.000000,45.0,0.369458,0.000000,1.000000,0.0,0.951984,102,NaN,NaN,2.079582,0.0,0.000000,NaN,0.000000,NaN,0.975764,NaN,0.933333,NaN,0.0,1.0,0.0,NaN,0.073967,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,1
30,146875,0,0.000000,NaN,0.0,NaN,NaN,0.000000,NaN,NaN,0.0,NaN,NaN,0.000000,0,50349.755383,NaN,NaN,NaN,0.000000,NaN,NaN,0.000000,NaN,NaN,NaN,0.000000,NaN,1,NaN,0,NaN,NaN,0.0,1.000000,1.000000,503.430000,444,NaN,0,NaN,0.0,0.0,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,0.259187,0.000000,NaN,NaN,NaN,0.0,0.978182,NaN,0.389747,0.000000,NaN,0.0,1.000000,0.277778,0.0,1.000000,0.0,NaN,20.00,NaN,20.0,0.394616,0.000000,NaN,0.0,0.998822,k01,NaN,NaN,1.528507,0.0,0.630223,NaN,0.609817,NaN,1.000000,NaN,1.000000,NaN,0.0,1.0,0.0,NaN,0.223881,0.0,1.0,0.430108,0.430108,0.0,0.0,0.000000,0.000000,0.000000,1
59,146914,0,0.000000,NaN,0.0,NaN,NaN,0.000000,NaN,NaN,0.0,NaN,NaN,0.000000,0,0.000000,NaN,NaN,NaN,0

In [16]:
X =  pd.concat([df_filtered_1, df_filtered_0], axis=0).reset_index(drop=True)
X

,ID,CR_PROD_CNT_IL,AMOUNT_RUB_CLO_PRC,APP_REGISTR_RGN_CODE,TURNOVER_DYNAMIC_IL_1M,CNT_TRAN_AUT_TENDENCY1M,SUM_TRAN_AUT_TENDENCY1M,AMOUNT_RUB_SUP_PRC,SUM_TRAN_AUT_TENDENCY3M,CLNT_TRUST_RELATION,REST_DYNAMIC_FDEP_1M,CNT_TRAN_AUT_TENDENCY3M,APP_MARITAL_STATUS,REST_DYNAMIC_SAVE_3M,CR_PROD_CNT_VCU,REST_AVG_CUR,CNT_TRAN_MED_TENDENCY1M,APP_KIND_OF_PROP_HABITATION,CLNT_JOB_POSITION_TYPE,AMOUNT_RUB_NAS_PRC,CLNT_JOB_POSITION,APP_DRIVING_LICENSE,TRANS_COUNT_SUP_PRC,APP_EDUCATION,CNT_TRAN_CLO_TENDENCY1M,SUM_TRAN_MED_TENDENCY1M,TRANS_COUNT_NAS_PRC,APP_TRAVEL_PASS,CR_PROD_CNT_TOVR,APP_CAR,CR_PROD_CNT_PIL,SUM_TRAN_CLO_TENDENCY1M,APP_POSITION_TYPE,TURNOVER_CC,TRANS_COUNT_ATM_PRC,AMOUNT_RUB_ATM_PRC,TURNOVER_PAYM,AGE,CNT_TRAN_MED_TENDENCY3M,CR_PROD_CNT_CC,SUM_TRAN_MED_TENDENCY3M,REST_DYNAMIC_FDEP_3M,REST_DYNAMIC_IL_1M,APP_EMP_TYPE,SUM_TRAN_CLO_TENDENCY3M,LDEAL_TENOR_MAX,LDEAL_YQZ_CHRG,CR_PROD_CNT_CCFP,DEAL_YQZ_IR_MAX,LDEAL_YQZ_COM,DEAL_YQZ_IR_MIN,CNT_TRAN_CLO_TENDENCY3M,REST_DYNAMIC_CUR_1M,REST_AVG_PAYM,LDEAL_TENOR_MIN,LDEAL_AMT_MONTH,APP_COMP_TYPE,LDEAL_GRACE_DAYS_PCT_MED,REST_DYNAMIC_CUR_3M,CNT_TRAN_SUP_TENDENCY3M,TURNOVER_DYNAMIC_CUR_1M,REST_DYNAMIC_PAYM_3M,SUM_TRAN_SUP_TENDENCY3M,REST_DYNAMIC_IL_3M,CNT_TRAN_ATM_TENDENCY3M,CNT_TRAN_ATM_TENDENCY1M,TURNOVER_DYNAMIC_IL_3M,SUM_TRAN_ATM_TENDENCY3M,DEAL_GRACE_DAYS_ACC_S1X1,AVG_PCT_MONTH_TO_PCLOSE,DEAL_YWZ_IR_MIN,SUM_TRAN_SUP_TENDENCY1M,DEAL_YWZ_IR_MAX,SUM_TRAN_ATM_TENDENCY1M,REST_DYNAMIC_PAYM_1M,CNT_TRAN_SUP_TENDENCY1M,DEAL_GRACE_DAYS_ACC_AVG,TURNOVER_DYNAMIC_CUR_3M,PACK,MAX_PCLOSE_DATE,LDEAL_YQZ_PC,CLNT_SETUP_TENOR,DEAL_GRACE_DAYS_ACC_MAX,TURNOVER_DYNAMIC_PAYM_3M,LDEAL_DELINQ_PER_MAXYQZ,TURNOVER_DYNAMIC_PAYM_1M,CLNT_SALARY_VALUE,TRANS_AMOUNT_TENDENCY3M,MED_DEBT_PRC_YQZ,TRANS_CNT_TENDENCY3M,LDEAL_USED_AMT_AVG_YQZ,REST_DYNAMIC_CC_1M,LDEAL_USED_AMT_AVG_YWZ,TURNOVER_DYNAMIC_CC_1M,AVG_PCT_DEBT_TO_DEAL_AMT,LDEAL_ACT_DAYS_ACC_PCT_AVG,REST_DYNAMIC_CC_3M,MED_DEBT_PRC_YWZ,LDEAL_ACT_DAYS_PCT_TR3,LDEAL_ACT_DAYS_PCT_AAVG,LDEAL_DELINQ_PER_MAXYWZ,TURNOVER_DYNAMIC_CC_3M,LDEAL_ACT_DAYS_PCT_TR,LDEAL_ACT_DAYS_PCT_TR4,LDEAL_ACT_DAYS_PCT_CURR,TARGET
0,146871,0,0.020267,NaN,0.0,1.000000,1.000000,0.111424,1.000000,mother,0.0,1.000000,NaN,0.994683,0,1049.544317,NaN,NaN,NaN,0.000000,NaN,NaN,0.379310,NaN,NaN,NaN,0.000000,NaN,1,NaN,0,NaN,NaN,0.0,0.000000,0.000000,0.000000,624,NaN,0,NaN,0.0,0.0,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,0.548043,0.000000,NaN,NaN,NaN,0.0,0.597867,0.363636,0.375444,0.000000,0.218070,0.0,NaN,NaN,0.0,NaN,0.0,NaN,20.00,0.218070,20.00,NaN,0.000000,0.363636,0.0,0.787479,o01,NaN,NaN,2.101087,0.0,0.000000,NaN,0.000000,NaN,0.655993,NaN,0.551724,NaN,0.0,1.0,0.0,NaN,0.479087,0.0,1.0,0.615412,0.615412,0.5,0.0,0.233333,0.233333,0.233333,1
1,146872,0,0.000000,NaN,0.0,NaN,NaN,0.016017,NaN,friend,0.0,NaN,NaN,0.000000,0,372.021867,NaN,NaN,NaN,0.000000,водитель-экспедитор,NaN,0.333333,NaN,NaN,NaN,0.000000,NaN,2,NaN,0,NaN,NaN,0.0,0.666667,0.983983,0.000000,276,NaN,0,NaN,0.0,0.0,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,0.508451,0.000000,NaN,NaN,NaN,0.0,0.929992,1.000000,0.366414,0.000000,1.000000,0.0,0.900000,0.500000,0.0,0.975369,0.0,NaN,45.00,1.000000,45.00,0.369458,0.000000,1.000000,0.0,0.951984,102,NaN,NaN,2.079582,0.0,0.000000,NaN,0.000000,NaN,0.975764,NaN,0.933333,NaN,0.0,1.0,0.0,NaN,0.073967,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,1
2,146875,0,0.000000,NaN,0.0,NaN,NaN,0.000000,NaN,NaN,0.0,NaN,NaN,0.000000,0,50349.755383,NaN,NaN,NaN,0.000000,NaN,NaN,0.000000,NaN,NaN,NaN,0.000000,NaN,1,NaN,0,NaN,NaN,0.0,1.000000,1.000000,503.430000,444,NaN,0,NaN,0.0,0.0,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,0.259187,0.000000,NaN,NaN,NaN,0.0,0.978182,NaN,0.389747,0.000000,NaN,0.0,1.000000,0.277778,0.0,1.000000,0.0,NaN,20.00,NaN,20.00,0.394616,0.000000,NaN,0.0,0.998822,k01,NaN,NaN,1.528507,0.0,0.630223,NaN,0.609817,NaN,1.000000,NaN,1.000000,NaN,0.0,1.0,0.0,NaN,0.223881,0.0,1.0,0.430108,0.430108,0.0,0.0,0.000000,0.000000,0.000000,1
3,146914,0,0.000000,NaN,0.0,NaN,NaN,0.000000,NaN,NaN,0.0,NaN,NaN,0.000000,0,0.000000,NaN,NaN,N

In [17]:
X.pop('ID')
X

,CR_PROD_CNT_IL,AMOUNT_RUB_CLO_PRC,APP_REGISTR_RGN_CODE,TURNOVER_DYNAMIC_IL_1M,CNT_TRAN_AUT_TENDENCY1M,SUM_TRAN_AUT_TENDENCY1M,AMOUNT_RUB_SUP_PRC,SUM_TRAN_AUT_TENDENCY3M,CLNT_TRUST_RELATION,REST_DYNAMIC_FDEP_1M,CNT_TRAN_AUT_TENDENCY3M,APP_MARITAL_STATUS,REST_DYNAMIC_SAVE_3M,CR_PROD_CNT_VCU,REST_AVG_CUR,CNT_TRAN_MED_TENDENCY1M,APP_KIND_OF_PROP_HABITATION,CLNT_JOB_POSITION_TYPE,AMOUNT_RUB_NAS_PRC,CLNT_JOB_POSITION,APP_DRIVING_LICENSE,TRANS_COUNT_SUP_PRC,APP_EDUCATION,CNT_TRAN_CLO_TENDENCY1M,SUM_TRAN_MED_TENDENCY1M,TRANS_COUNT_NAS_PRC,APP_TRAVEL_PASS,CR_PROD_CNT_TOVR,APP_CAR,CR_PROD_CNT_PIL,SUM_TRAN_CLO_TENDENCY1M,APP_POSITION_TYPE,TURNOVER_CC,TRANS_COUNT_ATM_PRC,AMOUNT_RUB_ATM_PRC,TURNOVER_PAYM,AGE,CNT_TRAN_MED_TENDENCY3M,CR_PROD_CNT_CC,SUM_TRAN_MED_TENDENCY3M,REST_DYNAMIC_FDEP_3M,REST_DYNAMIC_IL_1M,APP_EMP_TYPE,SUM_TRAN_CLO_TENDENCY3M,LDEAL_TENOR_MAX,LDEAL_YQZ_CHRG,CR_PROD_CNT_CCFP,DEAL_YQZ_IR_MAX,LDEAL_YQZ_COM,DEAL_YQZ_IR_MIN,CNT_TRAN_CLO_TENDENCY3M,REST_DYNAMIC_CUR_1M,REST_AVG_PAYM,LDEAL_TENOR_MIN,LDEAL_AMT_MONTH,APP_COMP_TYPE,LDEAL_GRACE_DAYS_PCT_MED,REST_DYNAMIC_CUR_3M,CNT_TRAN_SUP_TENDENCY3M,TURNOVER_DYNAMIC_CUR_1M,REST_DYNAMIC_PAYM_3M,SUM_TRAN_SUP_TENDENCY3M,REST_DYNAMIC_IL_3M,CNT_TRAN_ATM_TENDENCY3M,CNT_TRAN_ATM_TENDENCY1M,TURNOVER_DYNAMIC_IL_3M,SUM_TRAN_ATM_TENDENCY3M,DEAL_GRACE_DAYS_ACC_S1X1,AVG_PCT_MONTH_TO_PCLOSE,DEAL_YWZ_IR_MIN,SUM_TRAN_SUP_TENDENCY1M,DEAL_YWZ_IR_MAX,SUM_TRAN_ATM_TENDENCY1M,REST_DYNAMIC_PAYM_1M,CNT_TRAN_SUP_TENDENCY1M,DEAL_GRACE_DAYS_ACC_AVG,TURNOVER_DYNAMIC_CUR_3M,PACK,MAX_PCLOSE_DATE,LDEAL_YQZ_PC,CLNT_SETUP_TENOR,DEAL_GRACE_DAYS_ACC_MAX,TURNOVER_DYNAMIC_PAYM_3M,LDEAL_DELINQ_PER_MAXYQZ,TURNOVER_DYNAMIC_PAYM_1M,CLNT_SALARY_VALUE,TRANS_AMOUNT_TENDENCY3M,MED_DEBT_PRC_YQZ,TRANS_CNT_TENDENCY3M,LDEAL_USED_AMT_AVG_YQZ,REST_DYNAMIC_CC_1M,LDEAL_USED_AMT_AVG_YWZ,TURNOVER_DYNAMIC_CC_1M,AVG_PCT_DEBT_TO_DEAL_AMT,LDEAL_ACT_DAYS_ACC_PCT_AVG,REST_DYNAMIC_CC_3M,MED_DEBT_PRC_YWZ,LDEAL_ACT_DAYS_PCT_TR3,LDEAL_ACT_DAYS_PCT_AAVG,LDEAL_DELINQ_PER_MAXYWZ,TURNOVER_DYNAMIC_CC_3M,LDEAL_ACT_DAYS_PCT_TR,LDEAL_ACT_DAYS_PCT_TR4,LDEAL_ACT_DAYS_PCT_CURR,TARGET
0,0,0.020267,NaN,0.0,1.000000,1.000000,0.111424,1.000000,mother,0.0,1.000000,NaN,0.994683,0,1049.544317,NaN,NaN,NaN,0.000000,NaN,NaN,0.379310,NaN,NaN,NaN,0.000000,NaN,1,NaN,0,NaN,NaN,0.0,0.000000,0.000000,0.000000,624,NaN,0,NaN,0.0,0.0,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,0.548043,0.000000,NaN,NaN,NaN,0.0,0.597867,0.363636,0.375444,0.000000,0.218070,0.0,NaN,NaN,0.0,NaN,0.0,NaN,20.00,0.218070,20.00,NaN,0.000000,0.363636,0.0,0.787479,o01,NaN,NaN,2.101087,0.0,0.000000,NaN,0.000000,NaN,0.655993,NaN,0.551724,NaN,0.0,1.0,0.0,NaN,0.479087,0.0,1.0,0.615412,0.615412,0.5,0.0,0.233333,0.233333,0.233333,1
1,0,0.000000,NaN,0.0,NaN,NaN,0.016017,NaN,friend,0.0,NaN,NaN,0.000000,0,372.021867,NaN,NaN,NaN,0.000000,водитель-экспедитор,NaN,0.333333,NaN,NaN,NaN,0.000000,NaN,2,NaN,0,NaN,NaN,0.0,0.666667,0.983983,0.000000,276,NaN,0,NaN,0.0,0.0,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,0.508451,0.000000,NaN,NaN,NaN,0.0,0.929992,1.000000,0.366414,0.000000,1.000000,0.0,0.900000,0.500000,0.0,0.975369,0.0,NaN,45.00,1.000000,45.00,0.369458,0.000000,1.000000,0.0,0.951984,102,NaN,NaN,2.079582,0.0,0.000000,NaN,0.000000,NaN,0.975764,NaN,0.933333,NaN,0.0,1.0,0.0,NaN,0.073967,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,1
2,0,0.000000,NaN,0.0,NaN,NaN,0.000000,NaN,NaN,0.0,NaN,NaN,0.000000,0,50349.755383,NaN,NaN,NaN,0.000000,NaN,NaN,0.000000,NaN,NaN,NaN,0.000000,NaN,1,NaN,0,NaN,NaN,0.0,1.000000,1.000000,503.430000,444,NaN,0,NaN,0.0,0.0,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,0.259187,0.000000,NaN,NaN,NaN,0.0,0.978182,NaN,0.389747,0.000000,NaN,0.0,1.000000,0.277778,0.0,1.000000,0.0,NaN,20.00,NaN,20.00,0.394616,0.000000,NaN,0.0,0.998822,k01,NaN,NaN,1.528507,0.0,0.630223,NaN,0.609817,NaN,1.000000,NaN,1.000000,NaN,0.0,1.0,0.0,NaN,0.223881,0.0,1.0,0.430108,0.430108,0.0,0.0,0.000000,0.000000,0.000000,1
3,0,0.000000,NaN,0.0,NaN,NaN,0.000000,NaN,NaN,0.0,NaN,NaN,0.000000,0,0.000000,NaN,NaN,NaN,0.000000,NaN,NaN,0.000000,Na

In [18]:
procs = [Categorify, FillMissing, Normalize]
y_names = 'TARGET'
cont_names, cat_names = cont_cat_split(X, 1, dep_var=y_names)
y_block = CategoryBlock()
splits = RandomSplitter(valid_pct=0.2)(range_of(X))

In [19]:
cont_len, cat_len = len(cont_names), len(cat_names)
cont_len, cat_len

(91, 13)

In [20]:
to = TabularPandas(X, procs=procs, cat_names=cat_names, cont_names=cont_names,
                   y_names=y_names, y_block=y_block, splits=splits)
to.show()

,CLNT_TRUST_RELATION,APP_MARITAL_STATUS,APP_KIND_OF_PROP_HABITATION,CLNT_JOB_POSITION_TYPE,CLNT_JOB_POSITION,APP_DRIVING_LICENSE,APP_EDUCATION,APP_TRAVEL_PASS,APP_CAR,APP_POSITION_TYPE,APP_EMP_TYPE,APP_COMP_TYPE,PACK,AMOUNT_RUB_CLO_PRC_na,APP_REGISTR_RGN_CODE_na,CNT_TRAN_AUT_TENDENCY1M_na,SUM_TRAN_AUT_TENDENCY1M_na,AMOUNT_RUB_SUP_PRC_na,SUM_TRAN_AUT_TENDENCY3M_na,CNT_TRAN_AUT_TENDENCY3M_na,CNT_TRAN_MED_TENDENCY1M_na,AMOUNT_RUB_NAS_PRC_na,TRANS_COUNT_SUP_PRC_na,CNT_TRAN_CLO_TENDENCY1M_na,SUM_TRAN_MED_TENDENCY1M_na,TRANS_COUNT_NAS_PRC_na,SUM_TRAN_CLO_TENDENCY1M_na,TRANS_COUNT_ATM_PRC_na,AMOUNT_RUB_ATM_PRC_na,CNT_TRAN_MED_TENDENCY3M_na,SUM_TRAN_MED_TENDENCY3M_na,SUM_TRAN_CLO_TENDENCY3M_na,LDEAL_TENOR_MAX_na,LDEAL_YQZ_CHRG_na,DEAL_YQZ_IR_MAX_na,LDEAL_YQZ_COM_na,DEAL_YQZ_IR_MIN_na,CNT_TRAN_CLO_TENDENCY3M_na,LDEAL_TENOR_MIN_na,LDEAL_AMT_MONTH_na,CNT_TRAN_SUP_TENDENCY3M_na,SUM_TRAN_SUP_TENDENCY3M_na,CNT_TRAN_ATM_TENDENCY3M_na,CNT_TRAN_ATM_TENDENCY1M_na,SUM_TRAN_ATM_TENDENCY3M_na,DEAL_GRACE_DAYS_ACC_S1X1_na,AVG_PCT_MONTH_TO_PCLOSE_na,DEAL_YWZ_IR_MIN_na,SUM_TRAN_SUP_TENDENCY1M_na,DEAL_YWZ_IR_MAX_na,SUM_TRAN_ATM_TENDENCY1M_na,CNT_TRAN_SUP_TENDENCY1M_na,DEAL_GRACE_DAYS_ACC_AVG_na,MAX_PCLOSE_DATE_na,LDEAL_YQZ_PC_na,DEAL_GRACE_DAYS_ACC_MAX_na,LDEAL_DELINQ_PER_MAXYQZ_na,CLNT_SALARY_VALUE_na,TRANS_AMOUNT_TENDENCY3M_na,MED_DEBT_PRC_YQZ_na,TRANS_CNT_TENDENCY3M_na,LDEAL_USED_AMT_AVG_YQZ_na,LDEAL_USED_AMT_AVG_YWZ_na,AVG_PCT_DEBT_TO_DEAL_AMT_na,LDEAL_ACT_DAYS_ACC_PCT_AVG_na,MED_DEBT_PRC_YWZ_na,LDEAL_ACT_DAYS_PCT_TR3_na,LDEAL_ACT_DAYS_PCT_AAVG_na,LDEAL_DELINQ_PER_MAXYWZ_na,LDEAL_ACT_DAYS_PCT_TR_na,LDEAL_ACT_DAYS_PCT_TR4_na,LDEAL_ACT_DAYS_PCT_CURR_na,CR_PROD_CNT_IL,AMOUNT_RUB_CLO_PRC,APP_REGISTR_RGN_CODE,TURNOVER_DYNAMIC_IL_1M,CNT_TRAN_AUT_TENDENCY1M,SUM_TRAN_AUT_TENDENCY1M,AMOUNT_RUB_SUP_PRC,SUM_TRAN_AUT_TENDENCY3M,REST_DYNAMIC_FDEP_1M,CNT_TRAN_AUT_TENDENCY3M,REST_DYNAMIC_SAVE_3M,CR_PROD_CNT_VCU,REST_AVG_CUR,CNT_TRAN_MED_TENDENCY1M,AMOUNT_RUB_NAS_PRC,TRANS_COUNT_SUP_PRC,CNT_TRAN_CLO_TENDENCY1M,SUM_TRAN_MED_TENDENCY1M,TRANS_COUNT_NAS_PRC,CR_PROD_CNT_TOVR,CR_PROD_CNT_PIL,SUM_TRAN_CLO_TENDENCY1M,TURNOVER_CC,TRANS_COUNT_ATM_PRC,AMOUNT_RUB_ATM_PRC,TURNOVER_PAYM,AGE,CNT_TRAN_MED_TENDENCY3M,CR_PROD_CNT_CC,SUM_TRAN_MED_TENDENCY3M,REST_DYNAMIC_FDEP_3M,REST_DYNAMIC_IL_1M,SUM_TRAN_CLO_TENDENCY3M,LDEAL_TENOR_MAX,LDEAL_YQZ_CHRG,CR_PROD_CNT_CCFP,DEAL_YQZ_IR_MAX,LDEAL_YQZ_COM,DEAL_YQZ_IR_MIN,CNT_TRAN_CLO_TENDENCY3M,REST_DYNAMIC_CUR_1M,REST_AVG_PAYM,LDEAL_TENOR_MIN,LDEAL_AMT_MONTH,LDEAL_GRACE_DAYS_PCT_MED,REST_DYNAMIC_CUR_3M,CNT_TRAN_SUP_TENDENCY3M,TURNOVER_DYNAMIC_CUR_1M,REST_DYNAMIC_PAYM_3M,SUM_TRAN_SUP_TENDENCY3M,REST_DYNAMIC_IL_3M,CNT_TRAN_ATM_TENDENCY3M,CNT_TRAN_ATM_TENDENCY1M,TURNOVER_DYNAMIC_IL_3M,SUM_TRAN_ATM_TENDENCY3M,DEAL_GRACE_DAYS_ACC_S1X1,AVG_PCT_MONTH_TO_PCLOSE,DEAL_YWZ_IR_MIN,SUM_TRAN_SUP_TENDENCY1M,DEAL_YWZ_IR_MAX,SUM_TRAN_ATM_TENDENCY1M,REST_DYNAMIC_PAYM_1M,CNT_TRAN_SUP_TENDENCY1M,DEAL_GRACE_DAYS_ACC_AVG,TURNOVER_DYNAMIC_CUR_3M,MAX_PCLOSE_DATE,LDEAL_YQZ_PC,CLNT_SETUP_TENOR,DEAL_GRACE_DAYS_ACC_MAX,TURNOVER_DYNAMIC_PAYM_3M,LDEAL_DELINQ_PER_MAXYQZ,TURNOVER_DYNAMIC_PAYM_1M,CLNT_SALARY_VALUE,TRANS_AMOUNT_TENDENCY3M,MED_DEBT_PRC_YQZ,TRANS_CNT_TENDENCY3M,LDEAL_USED_AMT_AVG_YQZ,REST_DYNAMIC_CC_1M,LDEAL_USED_AMT_AVG_YWZ,TURNOVER_DYNAMIC_CC_1M,AVG_PCT_DEBT_TO_DEAL_AMT,LDEAL_ACT_DAYS_ACC_PCT_AVG,REST_DYNAMIC_CC_3M,MED_DEBT_PRC_YWZ,LDEAL_ACT_DAYS_PCT_TR3,LDEAL_ACT_DAYS_PCT_AAVG,LDEAL_DELINQ_PER_MAXYWZ,TURNOVER_DYNAMIC_CC_3M,LDEAL_ACT_DAYS_PCT_TR,LDEAL_ACT_DAYS_PCT_TR4,LDEAL_ACT_DAYS_PCT_CURR,TARGET
82551,friend,m,rent,self_empl,директор,y,h,n,y,top_manager,private,private,o01,False,False,True,True,False,True,True,True,False,False,True,True,False,True,False,False,True,True,True,True,True,True,True,True,True,True,True,False,False,False,False,False,False,True,False,False,False,False,False,False,True,True,False,True,True,False,True,False,True,False,True,False,False,False,False,False,False,False,False,0.0,0.009109,34.0,0.0,0.285714,0.277421,0.053070,0.705025,0.0

In [21]:
len(to.train),len(to.valid)

(168594, 42148)

In [22]:
y_train_distribution = to.train.ys.value_counts()
y_test_distribution = to.valid.ys.value_counts()
y_train_distribution, y_test_distribution

(TARGET
 0         145408
 1          23186
 Name: count, dtype: int64,
 TARGET
 0         36409
 1          5739
 Name: count, dtype: int64)

In [23]:
from imblearn.over_sampling import SMOTE

# Handling class imbalance with SMOTE
smote = SMOTE(random_state=42)
xs, ys = smote.fit_resample(to.train.xs, to.train.ys)

In [24]:
ys.value_counts()

TARGET
0         145408
1         145408
Name: count, dtype: int64

## 3. Model Development

### 3.1 - Baseline - naive classifier

- Use a naive classifier that predicts the most common class.

In [25]:
from sklearn.dummy import DummyClassifier
from sklearn.metrics import accuracy_score, roc_auc_score

from sklearn.dummy import DummyClassifier
from sklearn.metrics import accuracy_score, roc_auc_score


xs, valid_xs, ys, valid_ys = train_test_split(df.iloc[:,:-1], df['TARGET'], test_size=0.2)

dummy_clf = DummyClassifier(strategy="most_frequent")
dummy_clf.fit(xs, ys)
dummy_predictions = dummy_clf.predict(valid_xs)

# Evaluate the baseline model
baseline_accuracy = accuracy_score(valid_ys, dummy_predictions)
baseline_auc = roc_auc_score(valid_ys, dummy_predictions)

baseline_accuracy, baseline_auc

(0.91940933021763, 0.5)

In [26]:
update_summary_table(
    approach='Baseline (Naive Classifier)',
    lib='sklearn',
    accuracy=baseline_accuracy,
    auc=baseline_auc
)

,Approach,Library Used,Algorithm,Hyperparameters,Accuracy,AUC Score
0,Baseline (Naive Classifier),sklearn,Naive Classifier,N/A,0.919409,0.5
1,Random Forest,Scikit-learn,Random Forest,Optimized via Grid Search,To be filled,To be filled
2,Scikit-learn (MLPClassifier),Scikit-learn,MLPClassifier,Default / Custom Settings,To be filled,To be filled
3,Keras (TensorFlow),TensorFlow,Keras Neural Network,Custom Settings,To be filled,To be filled
4,TensorFlow,TensorFlow,TensorFlow Neural Network,Custom Settings,To be filled,To be filled
5,NumPy,NumPy,Custom Neural Network,Custom Implementation,To be filled,To be filled


1. **ROC Curve:** ROC stands for Receiver Operating Characteristic. It's a graph that shows the performance of a classification model at various classification thresholds. The x-axis represents the false positive rate (FPR), and the y-axis represents the true positive rate (TPR).

2. **AUC:** AUC is the area under the ROC curve. In simple terms, it's a measure of how well a model can distinguish between classes. The AUC value ranges from 0 to 1, where a higher value indicates better performance.

   - AUC = 1: Perfect classifier (100% true positive rate and 0% false positive rate).
   - AUC = 0.5: Classifier performs no better than random chance (the diagonal line).
   - AUC < 0.5: Worse than random chance.

So, when someone says the AUC is 0.8, for example, it means the model has a good ability to distinguish between positive and negative instances.

### 3.2 Random Forest:
- Implement a Random Forest classifier and use grid search for hyperparameter tuning.

Decision tree ensembles, as the name suggests, rely on decision trees. So let's start there! A decision tree asks a series of binary (that is, yes or no) questions about the data. After each question the data at that part of the tree is split between a "yes" and a "no" branch, as shown in figure below. After one or more questions, either a prediction can be made on the basis of all previous answers or another question is required.


In [ ]:
from sklearn.tree import DecisionTreeClassifier
from dtreeviz.trees import *
from IPython.display import Image, display_svg, SVG

from sklearn.metrics import accuracy_score, roc_auc_score

def m_auc(m, xs, y): return roc_auc_score(m.predict(xs), y)
def m_accuracy(m, xs, y): return accuracy_score(m.predict(xs), y)

m = DecisionTreeClassifier(max_leaf_nodes=4)
m.fit(to.train.xs, to.train.ys);

In [ ]:
draw_tree(m, to.train.xs, size=10, leaves_parallel=True, precision=2)

The top node represents the initial model before any splits have been done, when all the data is in one group. This is the simplest possible model. It is the result of asking zero questions and will always predict the value to be the average value of the whole dataset.

In this case, we can see it predicts a values of [261123, 23029] for the logarithm of the sales price. It gives a mean squared error of 0.48. The square root of this is 0.69. (Remember that unless you see m_rmse, or a root mean squared error, then the value you are looking at is before taking the square root, so it is just the average of the square of the differences.) We can also see that there are 404,710 auction records in this group—that is the total size of our training set. The final piece of information shown here is the decision criterion for the best split that was found, which is to split based on the coupler_system column.

In [ ]:
m_accuracy(m, to.train.xs, to.train.ys), m_accuracy(m, to.valid.xs, to.valid.ys)

In [ ]:
# m_auc(m, to.train.xs, to.train.ys), m_auc(m, to.valid.xs, to.valid.ys)

In [ ]:
from sklearn.utils.class_weight import compute_class_weight

kwargs = {'max_depth': 16, 'max_features': 'sqrt', 'n_estimators': 200}

def rf(xs, y, n_estimators=150, min_samples_leaf=10, **kwargs):
    return RandomForestClassifier(n_jobs=-1, min_samples_leaf=min_samples_leaf, oob_score=True, **kwargs).fit(xs, y)

In [ ]:
m = rf(to.train.xs, to.train.ys, **kwargs);

In [ ]:
m_accuracy(m, to.train.xs, to.train.ys), m_accuracy(m, to.valid.xs, to.valid.ys)

In [ ]:
m_auc(m, to.train.xs, to.train.ys), m_auc(m, to.valid.xs, to.valid.ys)

In [ ]:
def rf_feat_importance(m, df):
    return pd.DataFrame({'cols':df.columns, 'imp':m.feature_importances_}).sort_values('imp', ascending=False)

In [ ]:
fi = rf_feat_importance(m, to.train.xs)
fi[:10]

In [ ]:
def plot_fi(fi):
    return fi.plot('cols', 'imp', 'barh', figsize=(12,7), legend=False)

plot_fi(fi[:30]);

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

rf_model = RandomForestClassifier(random_state=42, n_jobs=-1)

# Define parameter grids using list comprehensions and numpy functions
n_estimators = [100, 200, 250]
max_features = ['log', 'sqrt']
max_depth = [8, 16, 24]
min_samples_split = [5, 10]

parameter_grid = {
    'n_estimators': n_estimators,
    'max_features': max_features,
    'max_depth': max_depth,
    'min_samples_split': min_samples_split,
}

grid_search = GridSearchCV(rf_model, parameter_grid, cv=3, scoring='roc_auc', n_jobs=-1)
grid_search.fit(to.train.xs, to.train.ys.iloc[:, 0])

best_rf_model = grid_search.best_estimator_

# Evaluate
rf_predictions = best_rf_model.predict(to.valid.xs)
rf_accuracy = accuracy_score(to.valid.ys.iloc[:, 0], rf_predictions)
rf_auc = roc_auc_score(to.valid.ys.iloc[:, 0], rf_predictions)

In [ ]:
best_rf_model = grid_search.best_estimator_

# Evaluate
rf_predictions = best_rf_model.predict(to.valid.xs)
rf_accuracy = accuracy_score(to.valid.ys.iloc[:, 0], rf_predictions)
rf_auc = roc_auc_score(to.valid.ys.iloc[:, 0], rf_predictions)
rf_accuracy, rf_auc

In [ ]:
grid_search.best_estimator_

In [ ]:
update_summary_table(
    approach='Random Forest',
    lib='sklearn',
    accuracy=rf_accuracy,
    auc=rf_auc
)

### 3.3 MLP Classifier Implementation

In [ ]:
from sklearn.neural_network import MLPClassifier

class MyMLPClassifier:
    def __init__(self, max_iter):
        # 
        self.model = MLPClassifier(**{'activation': 'relu', 'alpha': 0.01, 'hidden_layer_sizes': (50,50), 'learning_rate': 'adaptive', 'solver': 'adam'},  max_iter=max_iter, random_state=42, verbose=True)

    def train(self, X, y):
        self.model.fit(X, y)

    def evaluate(self, X, y):
        predictions = self.model.predict(X)
        accuracy = accuracy_score(y, predictions)
        auc = roc_auc_score(y, predictions)
        return accuracy, auc

mlp_classifier = MyMLPClassifier(max_iter=50)
mlp_classifier.train(to.train.xs, to.train.ys)
mlp_accuracy, mlp_auc = mlp_classifier.evaluate(to.valid.xs, to.valid.ys)
mlp_accuracy, mlp_auc

In [ ]:
update_summary_table(
    approach='Scikit-learn (MLPClassifier)',
    lib='sklearn',
    accuracy=mlp_accuracy,
    auc=mlp_auc
)

###  3.4 Keras

In [ ]:
import keras
import keras_tuner
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, Activation
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam

keras.utils.set_random_seed(42)
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
strategy = tf.distribute.get_strategy() # default strategy that works on CPU and single GPU
print("Number of accelerators: ", strategy.num_replicas_in_sync)

def build_model(hp, opt=None):
    input_features = to.train.xs.shape[1]
    with strategy.scope():
        model = Sequential()


        model.add(Dense(
            units=hp.Int('units_0' , min_value=256, max_value=2048, step=64),
            use_bias=True,
            activation='relu',
            input_shape=(input_features,)))
        model.add(Dropout(hp.Float(f"dropout_rate_0", min_value=0, max_value=0.6, step=0.2)))


        # Dense layers with Batch Normalization and ReLU
        for i in range(1, hp.Int('num_layers', min_value=6, max_value=12)):
            model.add(BatchNormalization())
            model.add(Dense(units=hp.Int('units_' + str(i), min_value=256, max_value=2048, step=512),
                            use_bias=True,
                            kernel_regularizer=l2(0.01)))
            model.add(Activation('relu'))
            model.add(Dropout(hp.Float(f"dropout_rate_{i}", min_value=0, max_value=0.6, step=0.2)))


        # Output layer
        model.add(Dense(1, activation='sigmoid'))

        # Compile the model
        optimizer = opt if opt is not None else Adam(hp.Float('lr', min_value=1e-4, max_value=1e-2, sampling='LOG'))
        model.compile(
            optimizer=optimizer,
            loss='binary_crossentropy',
            metrics=['AUC']
        )
    return model

In [ ]:
# Hyperparameter tuning setup
tuner = keras_tuner.RandomSearch(
    hypermodel=build_model,
    objective=keras_tuner.Objective("val_auc", direction="max"),
    max_trials=20,
    executions_per_trial=1,
    overwrite=False,
    directory="/kaggle/working/",
    project_name="hyperparameter_tuning",
    seed=42
)

# Start hyperparameter tuning
tuner.search_space_summary()

early_stopping = keras.callbacks.EarlyStopping(monitor='val_auc', patience=10, min_delta=1e-3, restore_best_weights=True)

# Assuming X_train, y_train, X_test, y_test, and BATCH_SIZE are defined and valid
tuner.search(to.train.xs, to.train.ys, validation_data=(to.valid.xs, to.valid.ys), epochs=70, batch_size=512, callbacks=[early_stopping])

# Display results
tuner.results_summary()

In [ ]:
best_hps = tuner.get_best_hyperparameters(1)[0]
best_hps.values

In [ ]:
best_hps_values = best_hps.values
file_path = 'best_hps_values.pkl'

with open(file_path, 'wb') as f:
    pickle.dump(best_hps_values, f)

print(f'Best hyperparameter values saved in {file_path}') 

In [ ]:
best_model = build_model(best_hps)

# Train the model for longer.

early_stopping = keras.callbacks.EarlyStopping(monitor='val_auc', patience=25, min_delta=5e-4, restore_best_weights=True)
history = best_model.fit(to.train.xs, to.train.ys, validation_data=(to.valid.xs, to.valid.ys), epochs=50, batch_size=512, callbacks=[early_stopping])

In [ ]:
(fig, ax) = plt.subplots(1, 1, figsize=(10, 8))

ax.plot(history.history["loss"], label="train_loss")
ax.plot(history.history["val_loss"], label="val_loss")
ax.set_title("Training Graphs")
ax.set_xlabel("Epochs")
ax.set_ylabel("Loss")
ax.set_ylim([0, 2.5])
ax.legend()

fig.tight_layout(pad=3.0)
fig.show()

In [ ]:
dev_predictions_proba = best_model.predict(to.valid.xs)
dev_predictions = (dev_predictions_proba > 0.5).astype('int')
keras_auc = roc_auc_score(to.valid.ys, dev_predictions_proba)
print('keras AUC:', keras_auc)
keras_acc = accuracy_score(to.valid.ys, dev_predictions)
print('keras ACC:', keras_acc)

keras_acc, keras_auc

In [ ]:
summary_table

In [ ]:
update_summary_table(
    approach='Keras (TensorFlow)',
    lib='TensorFlow',
    accuracy=keras_acc,
    auc=keras_auc
)

### 3.5 TensorFlow Approach

In [ ]:
xs, ys = tf.convert_to_tensor(to.train.xs, dtype=tf.float32), tf.convert_to_tensor(to.train.ys, dtype=tf.float32)
xs_valid, ys_valid = tf.convert_to_tensor(to.valid.xs, dtype=tf.float32), tf.convert_to_tensor(to.valid.ys, dtype=tf.float32)

xs.shape, ys.shape, xs_valid.shape, ys_valid.shape

In [ ]:
BATCH_SIZE = 512
train_ds = tf.data.Dataset.from_tensor_slices((xs, ys))
train_ds = train_ds.shuffle(buffer_size=xs.shape[0]).batch(BATCH_SIZE)

valid_ds = tf.data.Dataset.from_tensor_slices((xs_valid, ys_valid))
valid_ds = valid_ds.shuffle(buffer_size=xs_valid.shape[0]).batch(BATCH_SIZE)

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, optimizers, losses, metrics

In [ ]:
model = tf.keras.Sequential()
for i in range(best_hps_values['num_layers']):
    print(best_hps_values['units_%d' % (i)])
    model.add(BatchNormalization())
    model.add(layers.Dense(best_hps_values['units_%d' % (i)], activation=tf.nn.relu, kernel_regularizer=l2(0.01)))
    model.add(layers.Dropout(best_hps_values['dropout_rate_%d' % (i)]))
model.add(layers.Dense(1, activation=tf.nn.sigmoid))

# Compile the model
model.compile(optimizer=optimizers.Adam(learning_rate=best_hps_values['lr']),
                loss=losses.BinaryCrossentropy(),
                metrics=['accuracy', 'AUC'])

In [ ]:
model.build(input_shape=xs.shape)
model.summary()

In [ ]:
model.fit(xs, ys, batch_size=512, epochs=50, validation_data=(xs_valid, ys_valid))

In [ ]:
def loss_fn(y_true, y_pred):  # Use clearer variable names
    loss = tf.keras.losses.binary_crossentropy(y_true, y_pred)
    return tf.reduce_mean(loss)

def one_batch(model, x_batch, y_batch, loss_func, optimizer=None, is_train=False):  # Rename to train_step
    if is_train == True:
        with tf.GradientTape() as tape:
            y_pred = model(x_batch, training=True)
            loss = loss_func(y_batch, y_pred)

        gradients = tape.gradient(loss, model.variables)
        optimizer.apply_gradients(zip(gradients, model.variables))
    else:
        y_pred = model(x_batch, training=False)
        loss = loss_func(y_batch, y_pred)

    return loss

def train_model(model, train_dataset, val_dataset, optimizer, loss_func, epochs=10):  # Use 'val' for validation
    train_losses, val_losses = [], []

    for epoch in range(epochs):
        train_epoch_loss = tf.keras.metrics.Mean()  # Track epoch-wise loss
        val_epoch_loss = tf.keras.metrics.Mean()

        for x_batch, y_batch in train_dataset:
            train_batch_loss = one_batch(model, x_batch, y_batch, loss_func, optimizer=optimizer, is_train=True)
            train_epoch_loss.update_state(train_batch_loss)

        for x_batch, y_batch in val_dataset:
            val_batch_loss = one_batch(model, x_batch, y_batch, loss_func)
            val_epoch_loss.update_state(val_batch_loss)

        print(f"Epoch {epoch + 1}/{epochs} - Training loss: {train_epoch_loss.result():.4f}, Validation loss: {val_epoch_loss.result():.4f}")

        train_losses.append(train_epoch_loss.result())
        val_losses.append(val_epoch_loss.result())
    
    return train_losses, val_losses


In [ ]:
train_losses, val_losses = train_model(
    model,
    train_ds,
    valid_ds,
    optimizer=tf.optimizers.Adam(learning_rate=0.001,weight_decay=0.1),
    loss_func=loss_fn
)

In [ ]:
dev_predictions_proba = model(xs_valid, training=False)
dev_predictions = (dev_predictions_proba > 0.5).numpy().astype('int')
tensorflow_auc = roc_auc_score(ys_valid, dev_predictions_proba)
print('Tensorflow AUC:', tensorflow_auc)
tensorflow_acc = accuracy_score(ys_valid, dev_predictions)
print('Tensorflow ACC:', tensorflow_acc)

tensorflow_acc, tensorflow_auc

In [ ]:
update_summary_table(
    approach='TensorFlow',
    lib='TensorFlow',
    accuracy=tensorflow_acc,
    auc=tensorflow_auc
)

### Most sofisticated Neural network

In [ ]:
layers = [64, 128, 64]
metrics = [accuracy,RocAucBinary()]
dls = to.dataloaders(shuffle_train=True, device=torch.device('cuda'), bs=512)

# learn = tabular_learner(dls, layers=layers, metrics=metrics, config=tabular_config(ps=0.4))
learn = tabular_learner(dls, layers=layers, metrics=metrics, config=tabular_config(ps=0.4), wd=0.1)
learn.model.to('cuda')

learn.lr_find()

In [ ]:
learn.fit(n_epoch=4, lr=0.001)

In [ ]:
learn.lr_find()

In [ ]:
learn.fit_one_cycle(n_epoch=6, lr_max=1e-5)

In [ ]:
learn.model

In [26]:
import nn as nnx
from tensor import Tensor
from tqdm import tqdm
import matplotlib.pyplot as plt

In [27]:
class NeuralNetwork(nnx.Module):
    def __init__(self, input_shape, output_shape):
        super(NeuralNetwork, self).__init__()
        self.dense1 = nnx.Linear(in_features=input_shape, out_features=100)
        self.dense2 = nnx.Linear(100, 50)
        self.dense3 = nnx.Linear(50, 25)
        self.dense4 = nnx.Linear(25, output_shape)
        self.relu = nnx.ReLU()

    def forward(self, x):
        x = self.relu(self.dense1(x))
        x = self.relu(self.dense2(x))
        x = self.relu(self.dense3(x))
        x = self.dense4(x)
        return x

# Custom Dataset class
class MyDataset(nnx.Dataset):
    def __init__(self, X, y):
        self.X = Tensor(X)
        self.y = Tensor(y)
    
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, index):
        return self.X[index], self.y[index]

In [28]:
def fit(epochs, lr, model, loss_func, opt_fn, train_dl, valid_dl, patience=10, use_early_stoping=False):
    
    recorder = {'tr_loss': [], 'val_loss': [], 'tr_acc': [], 'val_acc': []}
    losses = [[], []]
    best_val_loss = float('inf')
    counter_early_stop = 0
    early_stop = False
    # fig, axs = plt.subplots(1, 1, figsize=(14, 7))
    # p = display(fig,display_id=True)

    opt = opt_fn(model.parameters(), lr=lr)
    
    for epoch in tqdm(range(epochs)):
        model.train()
        train_tot_loss, train_tot_acc, t_count = 0.,0.,0
        for xb,yb in train_dl:
            preds = model(xb)
            # import pdb; pdb.set_trace()
            loss = loss_func(preds, yb)
            loss.backward()
            opt.step()
            opt.zero_grad()

            # Calculate accuracy & loss
            predicted_labels = preds.argmax(axis=1)
            n = len(xb)
            t_count += n
            train_tot_loss += loss.item()*n
            train_tot_acc  += Tensor.accuracy(predicted_labels, yb).item()*n
            recorder['tr_loss'].append(loss.item())
            recorder['tr_acc'].append(Tensor.accuracy(predicted_labels, yb).item())
            losses[0].append(loss.item())

        model.eval()
        val_tot_loss, val_tot_acc,v_count = 0.,0.,0
        for xb,yb in valid_dl:
            preds = model(xb)

            pred_labels = preds.argmax(axis=1)
            n = len(xb)
            v_count += n
            val_tot_acc  += Tensor.accuracy(pred_labels, yb).item()*n
            val_tot_loss += loss_func(preds,yb).item()*n
            recorder['val_loss'].append(loss_func(preds,yb).item())
            recorder['val_acc'].append(Tensor.accuracy(pred_labels, yb).item())
            losses[1].append(loss_func(preds,yb).item())

            
        print(f"epoch {epoch + 1:02d}/{epochs:02d} - loss: {train_tot_loss/t_count:.4f} - acc: {train_tot_acc/t_count:.4f} - val_loss: {val_tot_loss/v_count:.4f} - val_acc: {val_tot_acc/v_count:.4f}")

        if use_early_stoping:
            if recorder['val_loss'][-1] < best_val_loss:
                best_val_loss = recorder['val_loss'][-1]
                counter_early_stop = 0
                # Save the best model
                model.save_weights(path='best_model.pth')
            else:
                counter_early_stop += 1
                if counter_early_stop >= patience:
                    print("Early stopping triggered")
                    early_stop = True
                    
            if early_stop:
                print("Stopped")
                break
            
    return recorder

In [29]:
X_tr, y_tr, X_val, y_val = map(Tensor, (to.train.xs, to.train.ys, to.valid.xs, to.valid.xs))

# Create the neural network
input_shape = X_tr.shape[1]  # Replace with the actual input shape
output_shape = 1  # Replace with the actual output shape

model = NeuralNetwork(input_shape, output_shape)

tr_ds = MyDataset(X_tr, y_tr)
val_ds = MyDataset(X_val, y_val)

# Creating the data loader
bs = 64
tr_dl = nnx.DataLoader(tr_ds, batch_size=bs)
val_dl = nnx.DataLoader(val_ds, batch_size=bs)

In [32]:
ds_path = 'ds.pkl'

with open(ds_path, 'wb') as f:
    pickle.dump((X_tr, y_tr, X_val, y_val), f)

In [30]:
xb, yb = next(iter(tr_dl))
xb.shape, yb.shape

((64, 163), (64, 1))

In [31]:
lr = 0.005
n_epochs = 25
recorder = fit(n_epochs, lr, model, nnx.BinaryCrossEntropyLoss(), nnx.SGD, tr_dl, val_dl)

  0%|     | 0/25 [00:00<?, ?it/s]


AssertionError: 

In [ ]:
# plt.figure(figsize=(10, 7))
plt.plot(recorder['tr_loss'], label='Training Loss')
plt.plot(recorder['val_loss'], label='Validation Loss', linestyle='--')
plt.xlabel('Iterations')
plt.ylabel('Loss')
plt.legend()
plt.title('Loss Curve')
plt.show()

plt.figure(figsize=(10, 7))
plt.plot(recorder['tr_acc'], label='Training Accuracy')
plt.plot(recorder['val_acc'], label='Validation Accuracy')
plt.xlabel('Iterations')
plt.ylabel('Accuracy')
plt.legend()
plt.title('Accuracy Curve')
plt.show()

# Save the model weights
model.save_weights(path='model_params.pkl')
print(f'>>> Saved model weights')






